## **LangChain**



- Framework para desarrollar apps basadas en LLMs

### Instalación

In [ ]:
! pip install langchain
! pip install --upgrade --quiet  langchain
! pip install langchain-community
! pip install langchainhub
! pip install langchain-openai
! pip install chromadb bs4

In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
key = os.getenv("OPENAI_API_KEY")

### **Componentes : LLM**

- OpenAI: Chat Model

Inicia el modelo

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0)


In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Write a recipe for the {ingredient}")

chain = prompt|llm

In [ ]:
# print(response)

- OpenAI: Embeddings Model

In [90]:
from langchain_openai import OpenAIEmbeddings

In [91]:
# Use OpenAIEmbeddings and size 3


In [ ]:
# Embed the text



In [ ]:
# Embed the query


### **Componentes : Chains**


### **Q&A : Indexing**

- Cargar texto (documentos, base de conocimiento)
- Trocear el texto para poder hacer llamadas a la API
- Almacenar el contenido

In [94]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import PyPDFDirectoryLoader

In [95]:
# Load the documents

loader = PyPDFDirectoryLoader("documents/")
docs = loader.load()

In [96]:
print(docs[0].page_content)

   
Contact
diana.bisbe@gmail.com
www.linkedin.com/in/
dianadiazbisbe  (LinkedIn)
Top Skills
Semantic Kernel
.NET Framework
Back-End Development
Certifications
Power BI Data Modeling with DAX
Global AI Bootcamp Participant
Intro to GitHub Copilot Course
Intro to Generative AI Course
BI Dashboards with Power BI
CourseDiana Diaz Bisbe
Cloud Solutions Developer @ ENCAMINA | Computer Science
Madrid, Community of Madrid, Spain
Summary
Diana Diaz Bisbe is a software engineer dedicated to the world of
artificial intelligence. 
Her passion for technology and her ability to solve complex technical
problems are evident in her work, where she primarily uses Python
and C# to develop impactful applications. Diana holds a degree in
Computer Science from the University of Central Florida. She is an
avid advocate for lifelong learning and stands out as a speaker in the
artificial intelligence community. She has held roles as a developer in
image processing and recognition projects, as well as in the c

In [97]:
# Chunk and embed the documents

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=chunks, embedding=OpenAIEmbeddings())


In [98]:
# Retrieve and generate using the relevant snippets of the blog.
prompt_text = '''You are an assistant for question-answering tasks. Use the following pieces of
        retrieved context to answer the question. If you don't know the answer, just say 
        that you don't know. Use three sentences maximum and keep the answer concise.
        Question: {question} 

        Context: {context} 

        Answer:
        '''
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_template(prompt_text)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [99]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
print (format_docs(docs))

In [101]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("Qué me puedes decir acerca del contexto?")